# Quadratic Sieve Implementation

## Preliminaries

Imports

In [ ]:
import math
import numpy as np

### Part 1.1: Strong Pseudoprime Test

A number of modules (below) require a means to quickly take some exponent mondulo some number...  
Notes on Fast Modular Exponentiation:  
* Use the property a * b mod (n) = a mod (n) * b mod (n)  
* Recursively compute: binary shifted right by one mod n

In [ ]:
# FAST MODULAR EXPONENTIATION
# b = base for exponentiation
# exp = exponent
# n = modulo
# returns result of fast modular exponentiation
def fastExp(b, e, m):
    res = 1
    while e > 0:
        if (e % 2):
            res = (res * b) % m
        e = e // 2
        b = (b * b) % m
    return res

In [ ]:
# Fast Modular Exponentiation Tests
print(fastExp(11, 13, 19))
print(fastExp(100, 2, 9991))

A few notes to consider when implementing the strong pseudoprime test:  
* There are no strong Carmichael Numbers - this was shown in class and homework  
* A composite number n is a strong pseudoprime to at most one quarter of all bases below n  

So, the test will be iterated over a number of bases to ensure (very high probability) primality  
For the test, there is no composite that passses the test for 2, 325, 9375, 28178, 450775, 9780504, and 1795265022  
* https://en.wikipedia.org/wiki/Strong_pseudoprime#cite_note-10  



In [ ]:
# STRONG PSEUDOPRIME TEST
# n = questionable prime
# bases = array of bases for test
# returns 0 on passed, -1 on failed test
def strongPseudo(n, bases):
    for i in range(len(bases)):
        t = n - 1
        while (not(t % 2)):
            res = fastExp(bases[i], t, n)
            if (res == 1):
                t = t // 2
                # Continue base checking
                continue
            elif (res == -1):
                # Assume probable pseudoprime of current base
                print("Strong Pseudoprime MSG: Probable Pseudoprime for base: {}".format(bases[i]))
                break
            else:
                # Number is composite - fails test
                print("Strong Pseudoprime MSG: {} is composite on base: {}".format(n, bases[i])) 
                return -1
    return 0

In [ ]:
# Strong Pseudoprime Tests
n = 3004879 
bases = [2, 325, 9375, 28178]
print(strongPseudo(n, bases))
print("Problem 1: {} is very unlikely to be prime (in fact its not)".format(n))

### Part 1.2: Trial Division

Notes on Trial Division:  
* Trial Division is used to factor out primes of some number, n, up to some max value.  
* The unfactored portion (after max) will be called 's'  
* The result of this module will be used in finding a root modulo p (Tonelli's Algorithm)

In [ ]:
# TRIAL DIVISION
# n = Number to divide
# parr = array of primes 
# returns arr (trial prime factorization exponents), n (AKA 's' the unfactored portion)
def trialDiv(n, parr):
    arr = [0] * len(parr)
    # Check each prime in prime array input
    for i in range(len(parr)):
        # Keep dividing prime out until no longer can
        while (True):
            if (n % parr[i] == 0):
                arr[i] = arr[i] + 1
                n = n // parr[i]
            else:
                break
    return arr, n

In [ ]:
# Trial Division Test
fb = [2, 3, 7, 11, 17, 23, 43, 59, 61]
print(trialDiv(2013, fb))

### Part 2.1: Inverse Modulo

In Tonelli's algorithm (see 2.2)   
Notes on Inverse Modulo:  
* Implement Extended Euclidean Algorithm for finding some inverse of a modulo n  


In [ ]:
# EUCLID's ALGORITHM
def myGCD(a, b):
    if b == 0:  
        return a
    return myGCD(b, a % b)

# EXTENDED EUCLIDEAN ALGORITHM
def extEuclid(a,b):
    n = b
    x_prime = 1
    x = 0
    #prevx, x = 1, 0
    while b:
        q = a // b
        # Update current iteration of inverse
        temp = x
        x = x_prime - (q * x)
        x_prime = temp
        # Update a & b
        temp1 = a
        a = b
        b = temp1 % b
    if (x_prime < 0):
        return n + x_prime
    return x_prime

In [ ]:
# Extended Euclidean Test
print(extEuclid(8, 193))
print("Test for Problem 2...")
print(extEuclid(1163, 1999))

In [ ]:
# GCD Test
print(myGCD(150, 140))
print(myGCD(1163, 1999))

### Part 2.2: Tonelli's Algorithm 

Calculating Jacobi symbols will be necessary for this module  
Notes on Jacobi Symbols:  
* This implementation of the Jacobi Symbols will indicate if there exists a square root congruent to some integer, 'a', and prime 'n'  
* A result of '1' indicates this square does exist, however Tonelli's algorithm is necessary for computing that square root (see Part 2.2)

In [ ]:
# JACOBI SYMBOLS
# n = integer in question
# p = prime modulus
# Returns 1 (on residue exists), -1 (on non-residue), 0 (p divides n)
def jacobi (a, n):
    j = 1
    # Main Loop
    while a != 0:
        # Loop for pulling out factors of two
        while a % 2 == 0:
            a = a // 2
            if ((n % 8 == 3) or (n % 8 == 5)):
                j = j * -1
        # Swap a, n
        temp = a
        a = n
        n = temp
        if ((a % 4 == 3) and (n % 4 == 3)):
            j = j * -1
        a = a % n
    if n == 1:
        return j
    return 0

In [ ]:
# Jacobi Test
a = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67]
n = 23587
for i in a:
    print("Jacobi Symbol {} / {} = {}".format(n, i, jacobi(n, i)))

Notes on Tonelli's Algorithm:  
This method is used for finding a root mod some prime, which will be very useful in the Quadratic Sieve   
* The following example detatils the steps  
  * Take x^2 = 9 mod 73  
  1. Check if 9 is a quadratic residue of 73 (via Jacobi Symbols)
  2. Use Trial Division (see 1.2) to obtain some s, t pair  
      * 73 - 1 = 72, 72 = 2^3 * 9 (s = 3, t = 9)  
  3. Find a non-residue (b mod p) via repeated jacobi symbols ((b/p) = -1) (b is new base - see below)
  4. Find an (even) i which satisfies (EQ: (a/(b^i))^(2^(s-k)*t) = 1 mod p), where 'a' is the square, 'p' is the prime, and 'b' is the base of i
  * b (inv)^i mod p  - > b (inv p) -> a * ans ^i -> ans ^ e -> check 1 mod p
      * Start with i = 2
      * Check if ik satisfies the EQ
            * If it does satisfy for remainder 1, then i(k+1) = i(k)    
             * O.W. i(k+1) = i(k) + 2^k
      * The above is computed for k = 1 -> s (see part 2)
  5. Take the final i(k = s) and compute: (b^(i/2)) * ((a/(b^i))^((t+1)/2)) mod p
  6. Pray 

In [ ]:
# TONELLI'S ALGORITHM 
# a = number 
# n = modulus 
# Returns a root mod n (r), o.w. -1
# NOTE: Remove prints to see i values
def tonelli(a, p):
    # Check Jacobi Symbol (a/p) = 1
    # if (jacobi(a, p) != 1):
    #     print("TONELLI's ALGORITHM ERROR: {} is not congruent to a quadratic residue mod {}".format(a, p))
    #     return -1
    # Get s, t
    s_arr, t = trialDiv(p - 1, [2])
    # print(s_arr, t)
    s = s_arr[0]
    # Find a non-residue 
    b = 2
    while (True):
        if (jacobi(b, p) == -1):
            break
        b = b + 1
    # Finding ik
    i_arr = []
    i = 2
    k = 1
    while (k <= s):
        # print("i = {}, k = {}".format(i, k))
        e = pow(2, (s-k)) * t
        # Get inverse a mod p
        if ((fastExp(a, e, p)) == fastExp(b, (i*e), p)):
        # if (extEuclid(a, p) == pow(b, (i))):
            # Current i works, append and continue
            i_arr.append(i)
            k = k + 1
            continue
        # Current i doesnt work, neeed to update
        # NOTE: One of the i values must work, so no need to check
        i = i + pow(2, k-1)
        i_arr.append(i)
        k = k + 1
    # print("TONELLI's ALGORITHM MSG: Array of i Values: \n{}".format(i_arr))
    # Compute final root
    alpha = a * pow(extEuclid(b, p), i)
    return (fastExp(b, (i//2), p) * fastExp(alpha, ((t+1)//2), p)) % p

In [ ]:
# Tonelli Test
print(tonelli(8, 193))
print("Test from Problem 2...")
print(tonelli(117, 3691))

In [ ]:
(117 * 117) % 3691

## Main Modules for Sieving

### Part 3: Gaussian Elimination Modulo 2

Notes on Gaussian Elimination:  
* METHOD:  
    * Start at column 0  
    * For row in row:  
    * if row, col value == 0  
        * loop over column values (below) in search of a leading zero (if nothing in that colmn then go to next column) - return its row index  
        * swap row with current one  
        * set current column if there were no more 1's in that column  
        * Otherwise, check all rows above and below  
        * Implement function to loop over above and subtract current row from it if a 1  
        * Handle out of bounds case (i.e., if it is a first row)  
        * Implement function to loop over below and subract current row from it if a 1  
        * Handle out of bounds case if last row   
    * Increment column  

In [ ]:
# Swap two numpy arrays
def swapRow(rref, row1, row2):
    # print("Swap Row {} and Row {}".format(row1, row2))
    rref[[row1, row2]] = rref[[row2, row1]]

# Subtract one numpy array from another (in mod 2)
def addRow(rref, redRow, leadRow):
    # print("Subtracting Lead Row {} from Row {}".format(leadRow, redRow))
    rref[redRow] = np.absolute(np.subtract(rref[redRow], rref[leadRow]))

# Search for 1's above the leading element in current row 
def searchAbove(rref, col, leadRow):
    # print("Serch Above {} in col {}".format(leadRow,col))
    # print("Column: {}".format(col))
    
    # Top-down row checking
    row = 0
    if (row == leadRow):
        # print("Search Failed: Lead Row {} Equal to Starting Search Row {}".format(leadRow, row))
        return -1
    while (row < leadRow):
        if (rref[row][col]):
            # print("Subtracting above")
            addRow(rref, row, leadRow)
            # print("After Subtract")
        row = row + 1
    return 0

# Search for 1's below the leading element in current row 
def searchBelow(rref, col, leadRow):
    # Down-Up row checking
    # print("Serch Below")
    # print("Column: {}".format(col))
    if (col >= rref.shape[1]):
        print("GAUSSIAN ELIMINATION MSG: MAX Columns in search below")
    row = rref.shape[0] - 1
    if (row == leadRow):
        print("GAUSSIAN ELIMINATION MSG: Search Failed: Lead Row {} Equal to Starting Search Row {}".format(leadRow, row))
        return -1
    while (row > leadRow):
        if (rref[row][col]):
            # print("Subtracting below")
            addRow(rref, row, leadRow)
            # print("After Subtract")
        row = row - 1
    return 0

# If leading element in row is not 1, search for new leading row 
def searchLead(rref, row, col):
    newRow, newCol = row + 1, col
    maxRow, maxCol = rref.shape[0], rref.shape[1]
    foundLead = 0
    # Ensure to not overflow dims of matrix
    if (newRow == maxRow or newCol == maxCol):
        print("GAUSSIAN ELIMINATION MSG: Maximum Row or Column Reached: [Row: {}| Column: {}]".format(maxRow, maxCol))
        return -1, -1
    while (newCol < maxCol):
        newRow = row + 1
        while (newRow < maxRow):
            # print("Row: {}".format(newRow))
            if (rref[newRow][newCol]):
                # Found a leading 1 => break and return 
                foundLead = 1
                break
            newRow = newRow + 1
        if (foundLead):
            break
        newCol = newCol + 1
    if (not(foundLead)):
        print("GAUSSIAN ELIMINATION MSG: New Leading Element not Found")
        return (newRow - 1), newCol
    return newRow, newCol

# GAUSSIAN ELIMINATION
# mat = matrix to be row reduced
# Returns rref, the input matrix in reduced row-echelon form
def gaussElim(mat):
    rref = np.asarray(mat)
    # print(rref)
    rows, cols = rref.shape[0], rref.shape[1]
    row, col = 0, 0
    while (row < rows):
        # Check Lead
        # print("BEGIN LOOP::::: Column: {}, Row: {}".format(col, row))
        if (rref[row][col] == 0):
            newRow, col = searchLead(rref, row, col)
            # print("New LEAD:: Column: {}, Row: {}".format(col, newRow))
            if (newRow == -1):
                # print("OVER")
                break
            elif(newRow != row):
                # print("Swap {} with {}".format(row, newRow))
                swapRow(rref, row, newRow)
                # print("After Swap")
                #print("New Pivot to check sub: [Row: {} | Col: {}]".format(row, col))
            else:
                print("GAUSSIAN ELIMINATION MSG: Error Finding Lead")
                break
        # Begin reduction
        # print("CURRENT ROW BEFORE SEARCHING: {}".format(row))
        if (row >= rows):
            break
        if (col >= cols):
            break
        searchAbove(rref, col, row)
        searchBelow(rref, col, row)
        # print("Row : {}, Col : {}".format(row, col))
        # print(rref)
        col = col + 1
        row = row + 1
        # print(rref)
        # Check column bounds
        if (col >= cols):
            break
    return rref

In [ ]:
# RREF Test
test1 = [[0, 0, 1], [0, 0, 1], [0, 1, 0]]
print("Easy Test Result...")
print(gaussElim(test1))
test2 = [[1, 0, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 0]]
print("Medium Test Result...")
print(gaussElim(test2))
print("Next Test for Problem 3...")
test3 = [[0,1,0,1,0,0,1,0,0,0],[0,0,1,1,0,1,0,0,1,1],[1,0,0,1,0,0,0,1,0,0],[1,0,1,0,0,1,0,1,1,1],[1,1,0,0,1,0,0,1,1,1],[1,0,0,0,1,1,1,1,1,0],[1,1,0,1,1,0,1,1,0,1],[1,0,0,1,0,0,0,0,0,0]]
print("Hard Test Result...")
print(gaussElim(test3))
print("Another Test...")
test4 = [[0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 1], [1, 0, 0, 1, 0, 1, 0], [1, 1, 0, 1, 1, 1, 0]]
print(gaussElim(test4))
print(gaussElim(test4).tolist())
print("Final Test...")
test = [[0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0], [1, 0,  0, 1, 0, 1, 1, 1, 0, 1, 1], [1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]]
print(gaussElim(test))


The Basis for the Solution Matrix Found in Gaussian Elimination  
* Find all free variables (or columns without leading 1's)  
* Test combinations of free variables  
    * For each free variable, set it to 1 and other 0 to find unique solution vector  
* Append Each vector and return as a matrix

In [ ]:
# FIND FREE VARIABLES  
# gauss = the rref matrix 
# Returns set, the set of independent vector solutions to the linear system
def find_free_vars(gauss):
    freeVars = [0] * gauss.shape[1]
    row, col = 0, 0
    maxRow, maxCol = gauss.shape[0], gauss.shape[1]
    # Search for leading zeros
    while (row < maxRow):
        while (col < maxCol):
            if (gauss[row][col]):
                # Found leading element
                freeVars[col] = 1
                col = col + 1
                break
            col = col + 1
        row = row + 1
    # Show all free variabls
    freeVariables = []
    for i in range(len(freeVars)):
        if freeVars[i] == 0:
            freeVariables.append(i) 
    if (len(freeVariables) > 0):
        return(freeVariables)
    print("No Free Vaiables Found...")
    return -1

In [ ]:
# Find free variables test
print("Test 1 Restuls...")
test1 = [[1, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]
test = np.asarray(test1)
print(find_free_vars(test))
test2 = [[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]
test = np.asarray(test2)
print("Test 2 Results...")
print(find_free_vars(test))

In [ ]:
# FIND BASIS VECTORS MOD 2
# rref = input matrix in rref form
# freeVars = Free vaiable columns in input matrix
# Returns basis, the array of basis vectors, or -1 if fail
def find_basis(rref, freeVars):
    basis = []
    cols = rref.shape[1]
    # Take each free column
    for i in range(len(freeVars)):
        freeCol = rref[:,freeVars[i]]
        basisVector = []
        k = 0
        # Insert free variables in column array
        for j in range(cols):
            if (j == freeVars[i]):
                # Set this basis vector element to 1
                basisVector.append(1)
            elif (j in freeVars):
                # A different free variable - set to 0
                basisVector.append(0)
            else:
                # Add the free column element 
                basisVector.append(freeCol[k])
                k = k + 1
        basis.append(basisVector)
    if (len(basis) != 0):
        return basis
    return -1

In [ ]:
# Basis Test
# First Test
print("Test 1 is Test from Problem 3...")
test1 = [[0,1,0,1,0,0,1,0,0,0],[0,0,1,1,0,1,0,0,1,1],[1,0,0,1,0,0,0,1,0,0],[1,0,1,0,0,1,0,1,1,1],[1,1,0,0,1,0,0,1,1,1],[1,0,0,0,1,1,1,1,1,0],[1,1,0,1,1,0,1,1,0,1],[1,0,0,1,0,0,0,0,0,0]]
print("Gaussian Elimination Hard Test Result...")
rref = gaussElim(test1)
print(rref)
print("Finding Free Variables")
frees = find_free_vars(rref)
for i in range(len(frees)):
    print("Free Variable {} is in column {}".format(i, frees[i]))
print("Total Number of Free Variables: {}".format(len(frees)))
print("Getting Basis...")
basis = find_basis(rref, frees)
print("Found the following Basis Vectors, with dimension {}".format(len(basis)))
for i in range(len(basis)):
    print(basis[i])
# Second Test
print("Test 2 is Test from rref example file...")
test2 =  [[0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0], [1, 0,  0, 1, 0, 1, 1, 1, 0, 1, 1], [1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]]
print("Gaussian Elimination Hard Test Result 2...")
rref = gaussElim(test2)
print(rref)
print("Finding Free Variables")
frees = find_free_vars(rref)
for i in range(len(frees)):
    print("Free Variable {} is in column {}".format(i, frees[i]))
print("Total Number of Free Variables: {}".format(len(frees)))
print("Getting Basis...")
basis = find_basis(rref, frees)
print("Found the following Basis Vectors, with dimension {}".format(len(basis)))
for i in range(len(basis)):
    print(basis[i])

### Part 4: Generate Factor Base

Before finding the factor base, need a means of getting all primes up to some integer, B  
Sieve of Eratosthenes:    
* Starting with p = 2, color all multiples of p up to b 
* For each iteration, the next non-colored integer is prime  
* Visual: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes


In [ ]:
# Sieve of Eratosthenes
# b = Limit of base
# Returns primes, an array of primes up to B
def eratosthenes(b):
    r = int(math.ceil(math.sqrt(b)))
    p = 2
    primes = []
    nums = [0] * b
    nums[0] = 1
    num = 1
    while (True):
        loc = p
        while (True):
            loc = p * num
            if (loc > b):
                break
            nums[loc-1] = 1
            num = num + 1
        primes.append(p)
        num = 1
        # Find next prime 
        loc = p
        found = 0
        while (loc <= b):
            # Number not yet colored - next prime
            if (nums[loc-1] == 0):
                p = loc
                found = 1
                break
            loc = loc + 1
        # Sought up to root and no more primes - done
        if (found == 0):
            break  
    return primes
    

In [ ]:
# Sieve of Eratosthenes Test
print(eratosthenes(30))
print(eratosthenes(70))

Notes on Factor Bases:  
* Find list of primes less than integer, B (via sieve of eratosthenes)  
* Ensure residues exist for n (number to be factored) and p (a prime element in the list)

In [ ]:
# FACTOR BASES
# b = Upper prime bound
# n = number to be factored
# Returns base, a list of primes
def factor_base(b, n):
    base = [2]
    primes = eratosthenes(b)
    for p in primes:
        if (p == 2):
            continue
        if (jacobi(n, p) == 1):
            base.append(p)
    return base

In [ ]:
# Factor Base Test
print(factor_base(30, 9991))
print(factor_base(70, 23587))
print("Problem 4...")
print(factor_base(200, 91))

### Part 5: Sieve

In [ ]:
# SIEVE
# B = Number of primes check for residue with some 'n'
# M = Number of r values to test
# n = Number to be factored 
# Threshold = the quantity sufficient of s(ri) to be included in the linear system
# Returns the final r values, the factor base, and tonelli results if success, otherwise -1 if failed to sieve enough r values
def quad_sieve(B, M, n, thresh):
    # Get a list of r values and a list of s(r) values
    rs = []
    s_rs = []
    r_1 = int(math.ceil(math.sqrt(n)))
    for i in range(r_1, r_1 + M + 2, 1):
        rs.append(i)
        s_rs.append(float(math.log(i*i - n)))
    # print("QUADRATIC SIEVE MSG: r Array: \n{} \ns(r) array: \n{}".format(rs, s_rs))
    # Get list of primes less than B that do have some root mod p
    factorBase = factor_base(B, n)
    print("QUADRATIC SIEVE MSG: Factor Base Size: {} \nPrime Elements: \n{}".format(len(factorBase), factorBase))
    # Compute roots from tonelli's algorithm
    tonelli_results = []
    for i in range(len(factorBase)):
        tonelli_results.append(tonelli(n, factorBase[i]))
    print("QUADRATIC SIEVE MSG: List of roots... \n{}".format(tonelli_results))
    # Run sieve
    for i in range(len(factorBase)):
        for j in range(len(rs)):
            # Check if r is congruent to current t
            res = rs[j] % factorBase[i]
            if ((res == tonelli_results[i]) or (res == (factorBase[i] - tonelli_results[i]))):
                s_rs[j] = s_rs[j] - float(math.log(factorBase[i]))
    # print("QUADRATIC SIEVE MSG: Final s(r) array: \n{}".format(s_rs))
    # Find final set of r values to solve linearly 
    final_rs = []
    for i in range(len(s_rs)):
        if (s_rs[i] <= thresh):
            final_rs.append(rs[i])
    # Ensure there are enough column vectors to find a factor
    if (len(final_rs) < (len(factorBase) + 1)):
        print("QUADRATIC SIEVE ERROR: Number of r values {} not sufficient for finding factors of {}, try higher threshold".format(len(final_rs), n))
        return -1, -1, -1
    # Found enough r values
    print("QUADRATIC SIEVE MSG: {} r values, {} factor base primes, {} tonellis roots".format(len(final_rs), len(factorBase), len(tonelli_results)))
    # print("QUADRATIC SIEVE MSG: Final r array: \n{}".format(final_rs))
    return final_rs, factorBase, tonelli_results 


In [ ]:
# Test Quadratic Sieve
Bs = [30, 70, 1000]
Ms = [40, 4900, 1000000]
thresh = [3, 1, 0.02]
n = [9991, 23587, 73868]
for i in range(len(n)):
    print("Test {}...".format(i + 1))
    if (i == 2):
        print("Test for Problem 5...")
    rs, fb, t = quad_sieve(Bs[i], Ms[i], n[i], thresh[i])
    print("r array: \n{}".format(rs))
    print("factor base: \n{}".format(fb))
    print("tonelli results: \n{}".format(t))



In [ ]:
# GENERATE SYSTEM OF PRIME POWERS
# rs = array of r values
# fb = factor base 
# n = the number to be factored
# Returns a matrix representing the linear system of prime powers for each r value found in sieve, or -1 if failure
def gen_system(rs, fb, n):
    sys = []
    # Generate prime factorization for each r
    for i in range(len(rs)):
        split = pow(rs[i], 2) - n
        parr, s = trialDiv(split, fb)
        if (s != 1):
            print("Error in system generation")
            return -1
        sys.append(parr)
    # Transpose system
    sys = np.asarray(sys)
    sys = sys.transpose()
    return sys

In [ ]:
# Test Generate System of Equations 
rs = [154, 157, 158, 159, 160, 170, 175, 182, 193, 197]
fb = [2, 3, 7, 11, 17, 23, 43, 59, 61]
n = 23587
print(gen_system(rs, fb, n))

In [ ]:
# Generate a mod 2 system of equations
def gen_mod2_sys(sys):
    # Create a mod 2 vector
    mod2Arr = [2] * len(sys[0])
    mod2Arr = np.asarray(mod2Arr)
    # Get mod 2 coefficients for each row in the system
    for i in range(sys.shape[0]):
        sys[i] = np.mod(sys[i], mod2Arr)
    return sys
    
# FIND PERFECT SQUARE
# sys = the matrix of prime powers for every r value (rows = factor base, columns = r's)
# basis = the basis vectors 
# rs = the r values from quadratic sieve
# fb = the factor base of primes
# Returns a basis vector, r value array, and a prime power vector on success, or -1 and -1 on failure to find a perfect square
def perfect_square(sys, rs, fb):
    # Get the mod 2 system 
    sysOrginal = sys.copy()
    sysMod2 = gen_mod2_sys(sys)
    # print("GENERATE PERFECT SQUARE MSG: Created Mod 2 System: \n{}".format(sys))
    # Row Reduce 
    rref = gaussElim(sysMod2)
    print("GENERATE PERFECT SQUARE MSG: The Row Reduced System: \n{}".format(rref))
    # Find free column vector offsets
    freeVars = find_free_vars(rref)
    print("GENERATE PERFECT SQUARE MSG: Free Column Vectors: \n{}".format(freeVars))
    # Get the basis vectors 
    basis = find_basis(rref, freeVars)
    print("GENERATE PERFECT SQUARE MSG: The Basis Vectors: \n{}".format(np.asarray(basis)))
    # Find a perfect square mod n
    primeCoeffs = []
    basisPassed = []
    finalRs = []
    for i in range(len(basis)):
        print("I: {}".format(i))
        parrs = []
        rVales = []
        square_coeffs = [0] * len(fb)
        # Check if vector's prime factorization mapping can be made into a square
        for j in range(len(basis[i])):
            if (basis[i][j]):
                rVales.append(rs[j])
                # Append prime factorization power coefficients to the prime powers array
                parrs.append(sysOrginal[:,j].tolist())
        print("GENERATE PERFECT SQUARE MSG: Prime Power Coefficents Found For Basis Vector: \n{}".format(basis[i]))
        # Verify square 
        # print(parrs)
        for k in range(len(parrs)):
            for v in range(len(parrs[k])):
                square_coeffs[v] = square_coeffs[v] + parrs[k][v]
        evenPower = 0
        for u in range(len(square_coeffs)):
            evenPower += square_coeffs[u]
        # All powers even => return prime powers vector
        print("EVEN: {}".format(evenPower))
        if (evenPower % 2 == 0):
            print("GENERATE PERFECT SQUARE MSG: Prime Power Coefficents Verified!: \nPowers: {} \nBasis Vector: {} \nr values: {}".format(square_coeffs, basis[i], rVales))
            basisPassed.append(basis[i])
            primeCoeffs.append(square_coeffs)
            finalRs.append(rVales)
        else:
            print("GENERATE PERFECT SQUARE MSG: Prime Power Coefficents Invalid: \nPowers: {} \nBasis Vector: {}".format(square_coeffs, freeVars[i]))
    if (len(primeCoeffs) > 0):
        return basisPassed, finalRs, primeCoeffs
    print("GENERATE PERFECT SQUARE ERROR: Failure To Find a Square On Basis: \n{}".format(basis))
    return -1, -1, -1

In [ ]:
# Perfect Square Test
print("Test 1...")
rs = [154, 157, 158, 159, 160, 170, 175, 182, 193, 197]
fb = [2, 3, 7, 11, 17, 23, 43, 59, 61]
n = 23587
sys = gen_system(rs, fb, n)
# print("Generated System...")
# print(sys)
bVector, rVales, psquares = perfect_square(sys, rs, fb)
print("Basis Vectors: {}".format(bVector))
print("r Vectors: {}".format(rVales))
print("Prime Powers: {}".format(psquares))
print("Test 2...")
rs = [100, 101, 104, 109, 115, 116, 118, 129, 137]
fb = [2, 3, 5, 7, 11, 19, 23]
n = 9991
sys = gen_system(rs, fb, n)
bVector, rVales, psquares = perfect_square(sys, rs, fb)
print("Basis Vectors: {}".format(bVector))
print("r Vectors: {}".format(rVales))
print("Prime Powers: {}".format(psquares))



Notes on Kraitchik's Algorithm:  
* Take r vector and get x mod n  
* Get root of prime vector, multiply out and get y mod n  
* Check if x^2 mod n == y^2 mod n  
* if so get gcd and check for if its a factor 


In [ ]:
# KRAITCHIK'S ALGORITHM
# rVectors = arrays of r's for x value
# pVectors = arrays of primes for y value
# n = the large integer to be factored
# Returns a factor on success, or -1 if failure to find a factor
def kraitchik(rVectors, pPowers, fb, n):
    # Find a factor for each vector 
    factors = []
    for i in range(len(rVectors)):
        # Get x
        x = 1
        for j in range(len(rVectors[i])):
            x = x * rVectors[i][j]
        x = x % n
        print("KRAITCHIK MSG: x = {} for vector {}".format(x, i))
        # Get y
        y = 1
        for j in range(len(pPowers[i])):
            y = y * (pow(fb[j], (pPowers[i][j] // 2)))
        y = y % n
        # Sanity check
        if (fastExp(x, 2, n) != fastExp(y, 2, n)):
            print("KRAITCHIK ERROR: {}^2 mod {} != {}^2 mod {}, continuing loop...".format(x, n, y, n))
            continue
        # GCD
        factor = myGCD(abs(x-y), n)
        if ((factor == 1) or (factor == n)):
            print("KRAITCHIK ERROR: {}, {} did not make a factor, continuing loop...".format(x, y))
        print("KRAITCHIK MSG: Found a Factor! {}".format(factor, i))
        factors.append(factor)
    if (len(factors) > 0):
        return factors
    print("KRAITCHIK ERROR: Did not find any factors for {}".format(n))
    return -1

## Examples of Factoring using the Quadratic Sieve

### Part 6: Wrap Up

In [ ]:
def main_module(n, B, M, T):
    # Check Strong Pseudoprime Test
    # bases = [2, 325, 9375, 28178]
    bases = [2]
    if (strongPseudo(n, bases) == 0):
        print("MAIN ERROR: Strong Pseudoprime Test did not show composite for {}".format(n))
        return -1
    print("MAIN MSG: {} very likely composite".format(n))
    # Choose sieve parameters 
    # B = int(math.floor(pow(math.e, (0.5 * math.sqrt(math.log(n) * math.log(math.log(n)))))))
    # M = pow(B, 2)
    # T = 0.
    # Sieve rs
    rs, fb, _ = quad_sieve(B, M, n, T)
    if (rs == -1):
        print("MAIN ERROR: Sieve Failure for n = {}".format(n))
    print("MAIN MSG: Quadratic Sieve Success")
    print(rs)
    print(fb)
    # Generate Linear System
    system = gen_system(rs, fb, n)
    print("MAIN MSG: System Generated: \n{}".format(system))
    # Find Perfect Squares 
    bVectors, rVectors, pVectors = perfect_square(system, rs, fb)
    if (bVectors == -1):
        print("MAIN ERROR: Finding Basis Vectors Failed for {}".format(n))
    print("MAIN MSG: {} Suitable Basis Vectors Found: \n{}\n{}\n{}".format(len(bVectors), bVectors, rVectors, pVectors))
    # Get Factors 
    factors = kraitchik(rVectors, pVectors, fb, n)
    if (factors == -1):
        print("MAIN ERROR: Finding Factors Failed for {}".format(n))
    print("MAIN MSG: Factors Found!")
    for i in range(len(factors)):
        if ((factors[i] != 1) and (factors[i] != n)):
            print("MAIN MSG: Factor Found: {}, {} / {} = {}".format(factors[i], n, factors[i], (n // factors[i])))
    print("MAIN MSG: {} Factored!".format(n))

In [ ]:
# Test main
print("Test 1...")
B = 70
M = 55
T = 4
main_module(23587, B, M, T)
print("\n\nTest 2...")
B = 1000
M = 1000000
T = 0.5
n = 7386829
main_module(n, B, M, T)

In [ ]:
# Test 
ns = [10378625636772128629]
p6 = 10378625636772128629
for i in range(len(ns)):
    if (ns[i] == p6):
        print("Test from problem 6: n = {}".format(ns[i]))
    b = int(math.floor(pow(math.e, (0.5 * math.sqrt(math.log(ns[i]) * math.log(math.log(ns[i])))))))
    print(b)
    m = b*b 
    print(n)
    t = math.log(b)
    main_module(ns[i], b, m, t)